In [ ]:
%%capture
!pip install git+https://github.com/vyperlang/titanoboa.git@master

In [ ]:
from google.colab import userdata
import boa

## **Setting Up the Infrastructure**

Before we can simulate some transactions, we need to initialize the necessary contract. For API keys, we can use the "Secrets" feature of Google Colab. An `RPC_ETHEREUM` including an HTTP key (e.g., from Alchemy) and an `ETHERSCAN_API_KEY` including an Etherscan API key need to be created before the notebook can be used.

Interacting with the contract is very simple when using titanoboa. The `from_etherscan` function lets us connect to the contracts.

`boa.env.fork` forks the desired chain for us.

Official Titanoboa Documentation: [https://titanoboa.readthedocs.io/en/latest/](https://titanoboa.readthedocs.io/en/latest/)


In [ ]:
boa.env.fork(userdata.get('RPC_ETHEREUM'))

ONEWAY_LENDING_FACTORY = '0xc67a44D958eeF0ff316C3a7c9E14FB96f6DedAA3'

factory = boa.from_etherscan(
    address=ONEWAY_LENDING_FACTORY,
    name='OneWayLendingFactory',
    api_key=userdata.get('ETHERSCAN_API_KEY')
)

factory

## **Changing Implementations**

Implementations can only be changed by the `admin` of the factory. To change the implementation, the admin (which is the CurveOwnershipAgent) needs to call the `set_implementations` function:

### **`set_implementations`**
`OneWayLendingVaultFactory.set_implementations(controller: address, amm: address, vault: address, pool_price_oracle: address, monetary_policy: address, gauge: address):`

Function to set new implementations. If a certain implementation should not be changed, `ZER0_ADDRESS` can be used as a placeholder.

| Input             | Type    | Description                           |
|-------------------|---------|---------------------------------------|
| controller        | address | New controller implementation.        |
| amm               | address | New amm implementation.               |
| vault             | address | New vault implementation.             |
| pool_price_oracle | address | New pool price oracle implementation. |
| monetary_policy   | address | New monetary policy implementation.   |
| gauge             | address | New gauge implementation.             |

In [ ]:
# lets check the current implementations
print("FACTORY IMPLEMENTATIONS BEFORE:")

print("controller implementation:", factory.controller_impl())
print("amm implementation:", factory.amm_impl())
print("vault implementation:", factory.vault_impl())
print("monetary policy implementation:", factory.monetary_policy_impl())

Let's impersonate the Factory admin, as this is the only address able to change the implementations and set some randomly generated addresses as the new implementations.

**NOTE:** The admin is the CurveOwnershipAdmin, which is controlled by the Curve DAO. When changing implementations in production, a DAO vote is required.

In [ ]:
ADMIN = factory.admin()

new_controller_impl = boa.env.generate_address()
new_amm_impl = boa.env.generate_address()
new_monetary_policy_impl = boa.env.generate_address()

with boa.env.prank(ADMIN):
    factory.set_implementations(
        new_controller_impl,
        new_amm_impl,
        "0x0000000000000000000000000000000000000000",
        "0x0000000000000000000000000000000000000000",
        new_monetary_policy_impl,
        "0x0000000000000000000000000000000000000000"
    )

assert new_controller_impl == factory.controller_impl()
assert new_amm_impl == factory.amm_impl()
assert new_monetary_policy_impl == factory.monetary_policy_impl()

print("Implementations have been changed successfully!")

*Now, let's check the implementations after changing them:*

In [ ]:
print("FACTORY IMPLEMENTATIONS AFTER:")

print("controller implementation:", factory.controller_impl())
print("amm implementation:", factory.amm_impl())
print("vault implementation:", factory.vault_impl())
print("monetary policy implementation:", factory.monetary_policy_impl())

**As shown, the desired implementation addresses of the OneWayLendingFactory were successfully changed.**
